#**Chat with Multiple PDF's Llama 2 + Pinecone + LangChain**

In [1]:
pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 36.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.1.0
    Uninstalling fsspec-2023.1.0:
      Successfully uninstalled fsspec-2023.1.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-hub-0.12.0
Note: you may need to restart the kernel to use updated packages.


#**Step: 01: Install All the Required Packages**

In [2]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate 
#!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: marshmallow
    Found existing installation: marshmallow 2.21.0
    Uninstalling marshmallow-2.21.0:
      Successfully uninstalled marshmallow-2.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.20.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 22.7 M

#**Step 2: Import All the Required Libraries**

In [3]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ku4ha9rf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ku4ha9rf
  Resolved https://github.com/huggingface/transformers to commit 6b7f8ff1f3db0b21afbedb322770c292bc8dedae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 71.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7888189 sha256=d4fe641c70d7c6efd0cf8586e6ada6383a9cd872e5433fd8a0c497a8c488f527
  Stored in directory: /tmp/pip-ephem-wheel-cache-e7lbqu0v/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Succes

In [4]:
pip install pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 64.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.2
    Uninstalling pydantic-1.9.2:
      Successfully uninstalled pydantic-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.1 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 1.10.8 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys


/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [10]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationalRetrievalChain

#**Step 06: Downlaod the Embeddings From the Hugging Face**

In [11]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [12]:
query_result = embeddings.embed_query("Hello World")

In [13]:
print("Length", len(query_result))

Length 384


#**Step 07: Initializing the Pinecone**

In [14]:
import pinecone      

pinecone.init(      
	api_key='75074dab-1f16-407a-a7f6-779bb1a05dc5',      
	environment='gcp-starter'      
)      
index = pinecone.Index('langchain2')

#**Step 08: Create Embeddings for each of the Text Chunk**

#If you already have an index, you can load it like this

In [15]:
docsearch = Pinecone.from_existing_index('langchain2', embeddings)


#**Step 09: Similarity Search**

In [16]:
query = "how do I make a dictionary in python?"

In [17]:
docs = docsearch.similarity_search(query, k=5)

In [18]:
docs

[Document(page_content='way i could have something like a list item in a dictionary something like this blah hi hello hello again and if you provide a solution could you tell me how i would need to change my code if it did require changes note i am using python 2 5'),
 Document(page_content='parrot any ideas how to create an elegant many to one dictionary in python thanks adam'),
 Document(page_content='combined_question: python creating a new variable from a dictionary not as straightforward as it seems i m trying to create a new variable that will consist of an existing dictionary so that i can change things in this new dictionary without it affecting the old one when i try this below which i think would be the obvious way to do this it still seems to edit my original dictionary when i make edits to the new one i have been searching for info on this but can t seem to find anything any info is appreciated newdictionary olddictionary'),
 Document(page_content='combined_question: is the

#**Step 10: Creating a Llama2 Model Wrapper**

In [19]:
from huggingface_hub import notebook_login
import torch

In [21]:
notebook_login()


In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [23]:
import torch
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True
                                             )

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [24]:

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [25]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})


#**Step 11: Create a Prompt Template**

In [26]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""


In [27]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [28]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [29]:
instruction = """
{context}

Question: {question}
"""

In [30]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [31]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<</SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [103]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [111]:
from langchain.chains.conversational_retrieval.prompts import QA_PROMPT

In [115]:
prompt=QA_PROMPT
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", output_key='answer', return_messages=True)

In [105]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

In [116]:
retriever = docsearch.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, 
                                           memory=memory, return_source_documents=True, 
                                           combine_docs_chain_kwargs={"prompt": prompt})

In [33]:
#qa_chain = RetrievalQA.from_chain_type(
    #llm=llm,
    #chain_type="stuff",
    #retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    #return_source_documents=True,
    #chain_type_kwargs={"prompt": prompt}
#)

In [117]:
result = qa("how do I read a csv?")

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [118]:
result


{'question': 'how do I read a csv?',
 'chat_history': [SystemMessage(content='')],
 'answer': " To read a CSV file in Python, you can use the `csv` module. Here's an example of how to do it:\n```\nwith open('file.csv', 'r') as f:\n    reader = csv.reader(f)\n    for row in reader:\n        print(row)\n```\nThis will read the entire file into a list of rows, where each row is a list of columns. You can access each column by its index in the list. For example, `row[0]` will give you the first column of the first row.\n\nAlternatively, you can use `csv.DictReader()` to read the file as a dictionary, where the columns are keys and the rows are values.\n```\nwith open('file.csv', 'r') as f:\n    reader = csv.DictReader(f)\n    for row in reader:\n        print(row)\n```\nThis will give you a dictionary where the keys are the column headers and the values are the values in each row.\n\nNote: The `csv` module also has other functions like `csv.writer` for writing CSV files, `csv.reader` for r